In [1]:
# Importing modules
try:
    import hsbalance as hs;
except ImportError:
    !pip install hsbalance
    import hsbalance as hs;

ModuleNotFoundError: No module named 'hsbalance'

# Introduction

This notebook discuss how the `hsbalance` package handles ill-conditioned planes problem

Darlow discussed in his paper `Darlow, M. S. "The identification and elimination of non-independent balance planes in influence coefficient balancing." Turbo Expo: Power for Land, Sea, and Air. Vol. 79603. American Society of Mechanical Engineers, 1982.` three test cases can be found in test cases in `/test/`

## Parameters

In [ ]:
ALPHA_math=[
                            ['1.41@45', '3.61@34', '3.61@34'],
                            ['3.16@72', '2.24@27', '2.24@27'],
                            ['2.83@45', '5@37', '5@37'],
                            ['3.16@18', '3.61@34', '4.47@27']]
A_math=[
                            ['3.16@72'], 
                            ['3.16@18'],
                            ['4.12@14'],
                            ['5.39@68']]

Convert to complex numbers (cartesian) form

In [ ]:
A = hs.convert_matrix_to_cart(A_math)
ALPHA = hs.convert_matrix_to_cart(ALPHA_math)
# A, ALPHA

Adding ALPHA

In [ ]:
alpha = hs.Alpha()
alpha.add(direct_matrix=ALPHA)

In [ ]:
alpha.check()

A warning is raised that plane[1] (the second plane, remember python starts counting from 0) is ill-conditioned.  
Ill-conditioned planes mean that they are algebraically depending on other planes. It means the ill-conditioned plane carries not new information to the system. When solving(especially with least squares method) the system tend to infiltrate (adding excessive masses at these planes)

In [ ]:
print(alpha)

## Solving with Least squares:

In [ ]:
model_LeastSquares = hs.LeastSquares(A, alpha, name='Least_squares') # Instantiate least square model

In [ ]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [ ]:
hs.convert_matrix_to_math(W_LeastSquares)

In [ ]:
residuals_LeastSquares = model_LeastSquares.expected_residual_vibration()
hs.convert_matrix_to_math(residuals_LeastSquares) # Expected residule vibrations

Root mean square error:

In [ ]:
rmse_LeastSquares = model_LeastSquares.rmse()
rmse_LeastSquares

### Discussion
The second plane is ill-conditioned as seen from the check, caused the second and third planes to increase the masses excessively.

In [ ]:
alpha.check(ill_condition_remove=True)

by turning ill_condition_remove boolean to True we remove the second plane (ill-conditioned)

In [ ]:
print(alpha)

## Solving with Least squares:

In [ ]:
model_LeastSquares = hs.LeastSquares(A, alpha, name='Least_squares') # Instantiate least square model

In [ ]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [ ]:
hs.convert_matrix_to_math(W_LeastSquares)

Remember those are weights for first and third planes as plane 2 has been removed by check.

In [ ]:
residuals_LeastSquares = model_LeastSquares.expected_residual_vibration()
hs.convert_matrix_to_math(residuals_LeastSquares) # Expected residule vibrations

Root mean square error:

In [ ]:
rmse_LeastSquares = model_LeastSquares.rmse()
rmse_LeastSquares

In [ ]:
print(model_LeastSquares.info())

### Discussion
by removing the second the correction weights at plane 3 has reduced to less than 4 times (from 5.14 kg to 1.14 kg). This causes a rise of RMSE and residual vibration on the other hand.   
It is left to the engineer judgment weather to remove (or constrain) the ill-conditioned planes or leave it to get the best RMSE possible.  
The target of this package is to give the engineer the best possible consulting service.